In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 5

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [3]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [5]:

# Define the MobileNet base model
base_model = MobileNet(
    input_shape=(img_height, img_width, 3),  # Adjust input shape
    include_top=False,  # Exclude the fully-connected layers
    weights='imagenet'  # Pre-trained on ImageNet
)

# Freeze the base model layers
base_model.trainable = False


In [6]:

# Create the classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(14, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




Epoch 1/5
235/235 [==============================] - 74s 184ms/step - loss: 1.8718 - accuracy: 0.4020 - val_loss: 1.1677 - val_accuracy: 0.5981
Epoch 2/5
235/235 [==============================] - 17s 71ms/step - loss: 1.2061 - accuracy: 0.6440 - val_loss: 1.0829 - val_accuracy: 0.6379
Epoch 3/5
235/235 [==============================] - 17s 72ms/step - loss: 0.9586 - accuracy: 0.7206 - val_loss: 0.7837 - val_accuracy: 0.7734
Epoch 4/5
235/235 [==============================] - 17s 71ms/step - loss: 0.8297 - accuracy: 0.7470 - val_loss: 0.7905 - val_accuracy: 0.7547
Epoch 5/5
235/235 [==============================] - 17s 71ms/step - loss: 0.7028 - accuracy: 0.7905 - val_loss: 0.7215 - val_accuracy: 0.7827


In [8]:
# predicting train dataset 
test_pridiction = model.predict(test_generator)

# true labels 
test_true_labels = test_generator.classes




86/86 [==============================] - 4s 20ms/step


In [9]:
print( test_pridiction.shape)
print( test_true_labels.shape)




(428, 14)
(428,)


In [ ]:
#calculating accuracy
accuracy = accuracy_score(test_true_labels, np.argmax(test_pridiction, axis=1))
print(accuracy)